<a href="https://colab.research.google.com/github/Maheenms/GoogleCoLab/blob/main/nlp_tokens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("NLPTokens").getOrCreate()

In [6]:
from pyspark.ml.feature import Tokenizer # it breaks the array into smaller pieces
from pyspark.sql.functions import col, udf #user defined functions . 
from pyspark.sql.types import IntegerType

In [4]:
# Create sample DataFrame
dataframe = spark.createDataFrame([
    (0, "Spark is great"),
    (1, "We are learning Spark"),
    (2, "Spark is better than hadoop no doubt")
], ["id", "sentence"])

In [8]:
# Show DataFrame
dataframe.show(truncate = False)

+---+------------------------------------+
|id |sentence                            |
+---+------------------------------------+
|0  |Spark is great                      |
|1  |We are learning Spark               |
|2  |Spark is better than hadoop no doubt|
+---+------------------------------------+



In [9]:
# Tokenize word
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
tokenizer

Tokenizer_2c96f9317eb5

In [10]:
# Show DataFrame
dataframe.show()
# Transform and show DataFrame
tokenized = tokenizer.transform(dataframe)
tokenized.show(truncate=False)

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|      Spark is great|
|  1|We are learning S...|
|  2|Spark is better t...|
+---+--------------------+

+---+------------------------------------+--------------------------------------------+
|id |sentence                            |words                                       |
+---+------------------------------------+--------------------------------------------+
|0  |Spark is great                      |[spark, is, great]                          |
|1  |We are learning Spark               |[we, are, learning, spark]                  |
|2  |Spark is better than hadoop no doubt|[spark, is, better, than, hadoop, no, doubt]|
+---+------------------------------------+--------------------------------------------+



In [12]:
# function will return the length of a list of words
def wordListLength(wordList):
  return len(wordList)

In [13]:
# call the udf function on the words column in the dataframe

 # udf takes 2 parameters -
 # input function, return data type of the function
 
tokenCounter = udf(wordListLength, IntegerType())

In [15]:
# apply the udf function
tokenized = tokenized.withColumn("word count", tokenCounter(col("words")))

tokenized.show(truncate=False)

+---+------------------------------------+--------------------------------------------+----------+
|id |sentence                            |words                                       |word count|
+---+------------------------------------+--------------------------------------------+----------+
|0  |Spark is great                      |[spark, is, great]                          |3         |
|1  |We are learning Spark               |[we, are, learning, spark]                  |4         |
|2  |Spark is better than hadoop no doubt|[spark, is, better, than, hadoop, no, doubt]|7         |
+---+------------------------------------+--------------------------------------------+----------+



In [16]:
# function that will count the number of vowels
def vowelCounter(wordList):
  # set up a total
  count = 0

  # use a loop to check for vowels
  for word in wordList:
    # for each word in the word array, do the following:
    # loop through each letter in the word
    for letter in word:
      # for each letter in each word, do the following:
      # check to see if the letter is a vowel - a, e, i, o, u
      if letter in ('a', 'e', 'i', 'o', 'u'):
        # add 1 to the count
        count += 1

  # once the loop finishes, return the count (integer)
  return count


In [17]:
vowelCounter = udf(vowelCounter, IntegerType())

tokenized = tokenized.withColumn("vowel count", vowelCounter(col("words")))

tokenized.show(truncate=False)

+---+------------------------------------+--------------------------------------------+----------+-----------+
|id |sentence                            |words                                       |word count|vowel count|
+---+------------------------------------+--------------------------------------------+----------+-----------+
|0  |Spark is great                      |[spark, is, great]                          |3         |4          |
|1  |We are learning Spark               |[we, are, learning, spark]                  |4         |7          |
|2  |Spark is better than hadoop no doubt|[spark, is, better, than, hadoop, no, doubt]|7         |11         |
+---+------------------------------------+--------------------------------------------+----------+-----------+

